## 단순 어휘 빈도에 근거한 키워드 추출
키워드를 특정 문헌의 내용을 요약적으로 표현하는 형태소의 집합이라고 정의할 때에 가장 먼저 시도할 수 있는 키워드 추출 방법은 형태소 빈도에 의한 방법이다.
즉, 특정 문헌에서 많이 나타나는 형태소가 그 문헌의 내용을 나타낼 가능성이 높다는 가정을 하는 것이다.

다음 스크립트는 문헌별 단순 어휘 빈도에 근거한 문헌별 키워드 추출을 수행한다.

>문헌별 단순 어휘 빈도를 문헌 정보학에서는 용어로 용어 빈도(term frequency)라 부른다.

일반명사(NNG), 고유명사(NNP), 어근(XR) 형태소를 추출한 뒤 이들의 용어 빈도를 측정한 다음 상위 10개만 키워드로 추출한다. 

## 용어 빈도 역문헌 빈도에 근거한 키워드 추출
위에서 보인 용어 빈도에 근거한 키워드 추출 결과는 제법 유용하다. 
하지만 어떤 키워드들은 해당 논문의 내용을 나타낸다고 할 수는 있지만 너무 흔하게 쓰이는 용어여서 해당 논문만의 특징적인 내용을 나타내기에는 부적합할 수도 있다. 
즉, 용어 빈도에 근거해 추출한 키워드들 가운데에는 특정성(specificity)이 부족한 것들이 있다.
특정성이 높은 용어들의 특징은 상대적으로 드물게 쓰인다는 것이다.
다른 말로 하면, 넓게 쓰이지 않는다고 할 수 있으며, 결국 어떤 용어가 하나의 문헌에서도 많이 사용되고 그 문헌이 속한 전체 문헌 집합에 속한 다수의 다른 문헌들에서 널리 쓰였다면 이 용어는 특정성이 떨어지는 용어이다.
이 때 특정 용어가 사용된 문헌의 수를 문헌 빈도(DF: document frequency)라고 부르며, 이의 역에 해당하는 역문헌 빈도(IDF: inverse document frequency)도 정의할 수 있다. 
수식으로는 다음과 같이 표현할 수 있다.

$$
    \operatorname{idf}(t, D) = \log\frac{|D|}{|\{d \in D: t \in d\}|}
$$

* $|D|$: 전체 문헌 집합 $D$의 크기, 곧 전체 문헌의 수
* $|\{d \in D: t \in d\}|$: 용어 $t$가 포함된 문헌의 수

위의 수식의 의미는 주어진 용어 $t$와 전체 문헌 집합 $D$에 대한 역문헌 빈도는 전체 문헌의 수를 주어진 용어가 출현한 문헌들의 수, 즉 문헌 빈도로 나눈 값에 로그를 취한 값으로 정의한다는 것이다.
주어진 용어 $t$가 출현한 문서의 수가 많으면 많을수록 역문헌 빈도의 값이 작아지며, 그러한 문서의 수가 적으면 적을수록 역문헌 빈도의 값이 커진다.
즉, 특정성이 커진다.
나누기 연산을 통해 얻어진 값에 로그를 취하여 스케일링을 하면 값이 평탄화가 이루어진다.
한편, 어떤 용어의 문헌 빈도가 0일 경우에 대해서도 역문헌 빈도를 정의해야 할 수 있기 때문에 실제로는 다음과 같이 최소 문헌 빈도를 1로 설정한 값이 많이 사용된다.

$$
    \operatorname{idf}(t, D) = \log\frac{|D|}{|\{d \in D: t \in d\}|+1}
$$

실제 자료 처리에 있어서는 용어 빈도도 다음과 같이 평탄화하여 사용한다.

$$
    \operatorname{tf}(t, d) = \log(\operatorname{tf}(t, d)+1)
$$

TFIDF를 구하는 위의 과정을 앞서 용어 빈도에 근거한 키워드 추출 스크립트에 추가하여 구현한 스크립트의 소스는 다음과 같다.
추출한 키워드는 다수의 문헌이 포함된 문헌 집합 내에서 특정 문헌을 향한 특정성, 즉 역문헌 빈도를 이용하여 용어 빈도를 조정함으로 하여 추출된 것이다.

In [1]:
# 용어 빈도 역문헌 빈도에 의한 키워드 추출

import ujson
import math
from collections import Counter

FEATURE_POSES = ["NNG", "NNP", "NR", "NP", "XR", "MAG", "VA", "VV"]
MA_KEYS = ["question_ma", "answer_ma"]
TOP_N = 20


def read_documents(input_file):
    """형태소 분석 문서 집합을 읽어서 돌려준다."""
    
    print_log("Reading input file.")
    documents = []

    for line in input_file:
        json_obj = ujson.loads(line)

        if not check_ma_keys(json_obj):
            continue

        morphs = []

        for ma_key in MA_KEYS:
            for morph_lex, morph_cat in json_obj[ma_key]:
                if morph_cat not in FEATURE_POSES:
                    continue
                    
                morphs.append(morph_lex)
        #print("문답[{}] 키워드 추출대상 단어[{}]".format(json_obj["no"], morphs))
        document = {
            "no": json_obj["no"],
            "question": json_obj["question"],
            "answer": json_obj["refine_answer"], 
            "morphs": morphs
        }
        documents.append(document)

    return documents


def check_ma_keys(json_obj):
    """JSON 객체에 형태소 분석 키가 존재하는지 검사한다."""
    
    for ma_key in MA_KEYS:
        if ma_key not in json_obj or not json_obj[ma_key]:
            return False
        
    return True


def get_term_freq_counters(documents):
    """문서 집합으로부터 문서별 용어 빈도 계수 결과를 생성한다."""
    
    print_log("Getting term frequency counters.")
    term_freq_counters = []

    for document in documents:
        term_freq_counter = Counter(document["morphs"])
        term_freq_counters.append(term_freq_counter)

    return term_freq_counters


def extract_tf_keywords(term_freq_counters):
    """용어 빈도에 의거하여 키워드를 추출한다."""
    
    print_log("Extracting TF keywords.")
    tf_keywords = []

    for term_freq_counter in term_freq_counters:
        keywords = []
        
        for term, term_freq in term_freq_counter.most_common(TOP_N):
            keywords.append(term)
            
        tf_keywords.append(keywords)

    return tf_keywords


def get_doc_freq_counter(documents):
    """주어진 문서 집합으로부터 문헌 빈도를 계수하여 돌려준다."""
    
    print_log("Getting document frequency counter.")
    doc_freq_counter = Counter()

    for document in documents:
        doc_freq_counter.update(set(document["morphs"]))

    return doc_freq_counter


def get_tfidf_counters(term_freq_counters, doc_freq_counter):
    """주어진 용어 빈도, 문헌 빈도 계수 결과에서 tfidf를 얻어서 돌려준다."""

    tfidf_counters = []
    num_docs = len(term_freq_counters)

    for term_freq_counter in term_freq_counters:
        tfidf_counter = Counter()

        for term, term_freq in term_freq_counter.items():
            tf = math.log(term_freq + 1)
            df = doc_freq_counter[term] + 1
            idf = math.log(num_docs / df)
            tfidf = tf * idf
            tfidf_counter[term] = tfidf

        tfidf_counters.append(tfidf_counter)

    return tfidf_counters


def extract_tfidf_keywords(tfidf_counters):
    """tfidf에 의거한 키워드를 추출한다."""
    
    print_log("Extracting TFIDF keywords.")
    tfidf_keywords = []

    for tfidf_counter in tfidf_counters:
        keywords = []

        for term, tfidf in tfidf_counter.most_common(20):
            keywords.append(term)

        tfidf_keywords.append(keywords)

    return tfidf_keywords


def write_keywords(output_file, documents, tf_keywords, tfidf_keywords):
    """출력 파일에 문헌별 키워드를 기록한다."""
    
    print_log("Writing keywords to output file.")

    for document, tf_keywords, tfidf_keywords in zip(documents, tf_keywords,
                                                     tfidf_keywords):
        print("질문[{}]: {}".format(document["no"],document["question"]), file=output_file)
        print("답변: {}".format(document["answer"]), file=output_file)
        print("TF키워드: {}".format(", ".join(tf_keywords)), file=output_file)
        print("TFIDF키워드: {}".format(", ".join(tfidf_keywords)),
              file=output_file)
        print("=" * 60, file=output_file)

        
def print_log(msg):
    """주어진 로그 메시지를 출력한다."""
    
    print(msg, flush=True)
    

def main():
    """tfidf에 의거하여 키워드를 추출한다."""
    
    input_file_name = "data/catechism/catechism.ma.txt"
    output_file_name = "data/catechism/catechism.kw.tfidf.txt"
    doc_no = 0
    with open(input_file_name, "r", encoding="utf-8") as input_file, \
            open(output_file_name, "w", encoding="utf-8") as output_file:
        documents = read_documents(input_file)
        term_freq_counters = get_term_freq_counters(documents)
        tf_keywords = extract_tf_keywords(term_freq_counters)
        doc_freq_counter = get_doc_freq_counter(documents)
        tfidf_counters = get_tfidf_counters(term_freq_counters, doc_freq_counter)
        tfidf_keywords = extract_tfidf_keywords(tfidf_counters)
        write_keywords(output_file, documents, tf_keywords, tfidf_keywords)

    for document, tf_keywords, tfidf_keywords in zip(documents, tf_keywords, tfidf_keywords):
        print("질문[{}]   : {}".format(document["no"],document["question"]))
        print("답변[{}]   : {}".format(document["no"],document["answer"]))
        print("TF키워드   : {}".format(", ".join(tf_keywords)))
        print("TFIDF키워드: {}".format(", ".join(tfidf_keywords)))
        print("*"*100)
        
#        
# 실행
#
main()

Reading input file.
Getting term frequency counters.
Extracting TF keywords.
Getting document frequency counter.
Extracting TFIDF keywords.
Writing keywords to output file.
질문[1]   : 사람의 제일 되는 목적이 무엇인가?
답변[1]   : 사람의 제일 되는 목적은 하나님을 영화롭게 하는 것과 영원토록 그를 즐거워하는 것이다.
TF키워드   : 사람, 제일, 되, 목적, 무엇, 하나님, 영화, 하, 영원, 그, 즐거워하
TFIDF키워드: 목적, 제일, 즐거워하, 되, 영화, 영원, 사람, 그, 하, 하나님, 무엇
****************************************************************************************************
질문[2]   : 하나님께서 무슨 규칙을 우리에게 주시어 어떻게 자기를 영화롭게 하고 즐거워할 것을 지시하셨는가?
답변[2]   : 신구약 성경에 기재된 하나님의 말씀은 어떻게 우리가 그를 영화롭게 하고 즐거워할 것을 지시하는 유일한 규칙이다.
TF키워드   : 하나님, 규칙, 우리, 어떻, 영화, 하, 즐거워하, 지시, 자기, 신구약, 성경, 기재, 말씀, 그, 유일한
TFIDF키워드: 규칙, 지시, 즐거워하, 영화, 신구약, 기재, 성경, 유일한, 어떻, 자기, 말씀, 그, 우리, 하, 하나님
****************************************************************************************************
질문[3]   : 성경이 제일 요긴하게 교훈하는 것이 무엇인가?
답변[3]   : 성경이 제일 요긴하게 교훈하는 것은 사람이 하나님에 대하여 어떻게 믿을 것과 하나님께서 사람에게 요구하시는 본분이다.
TF키워드   : 성경, 제일, 요긴, 교훈, 사람, 하나님, 무엇